### <font face="微软雅黑" color=#A52A2A > 1.为什么Bagging降方差，boosting降偏差

偏差是指预测值与真实值之间的差距，方差是指算法在相同样本数量的不用训练集上的方差。boosting是通过不断的迭代对y值进行逼近，每一轮的迭代都会针对损失函数降低误差，所以偏差越来越小；  
而bagging是通过不断重采样，生成多个训练集，在每个训练集中训练一个学习器，最后整合的过程。比如在数量为m的初始训练集上，每次采样都放回，采样m次后得到一个采样集，训练一个学习器。bagging并不会显著的降低bias，因为在不同的训练集上用的算法其实是一样的。它其实只是增强了模型的鲁棒性，通过在不同的训练集上进行训练，数据扰动带来的误差会被降低。

### <font face="微软雅黑" color=#A52A2A >2.随机森林与GBDT区别

随机森林是bagging的一个扩展变体。RF在以决策树为基学习器构建bagging集成的基础上，引入了随机属性的选择。传统的决策树在选择划分属性的时候是在当前节点的属性集合中选择一个最优属性；而在RF中，对基学习器的每个节点，先从该节点的属性集合中随机选择一个包含k个属性的子集，再从子集中选择一个最优属性进行划分。如果k=d，那么基学习器和传统决策树相同；如果k=1，那么就是随机选择属性进行划分。RF的基学习器的多样性不仅来自样本扰动，而且来自属性扰动，最终泛化性能可以通过个体学习器的差异增加得到进一步提升。

GBDT是一个boosting的算法，每一轮迭代都会生成一个决策树模型，最后的预测结果是把所有的弱学习器的结果相加。如果预测值看成自变量，那么boosting的过程就是损失函数在做梯度下降的过程，每一轮迭代但是在拟合伪残差。

### <font face="微软雅黑" color=#A52A2A > 3. 为什么xgboost效果不如随机森林

xgboost是boosting的过程，迭代过程主要是降低偏差，让预测更接近真实值。不如RF的可能原因有很多，如果xgboost训练的轮数不够，或者是其他一些超参数的调节不优，就会影响最后的结果。如果在调整的比较好的情况下，又是用的同样一组训练集，那么更可能的原因是xgboost产生了overfitting，产生了比较大的方差，导致在训练集的误差很小，但是在测试集上的误差比较大。

对于RF而言，由于它是基于bagging的集成算法，它会在使用重采样的方式生成多个训练集，然后训练多个学习器，最后取平均作为结果。另外，RF在选择分裂的属性的时候也加入了随机性，这样的方式更robust，所以能够比较好的降低方差。

综上所述，xgboost不如随机森林的原因可能是训练集和测试集的分布出现了不一样的特点，导致boosting方法出现了泛化误差比较大的情况。

### <font face="微软雅黑" color=#A52A2A > 4. xgb对gbdt有什么提升，lgb对xgb有什么提升，为什么有了xgb还要使用lgb

xgboost是高阶版的GBDT，它们之间主要有着以下不同：  
原理上：  
1. 传统GBDT主要采用了cart作为基学习器，而xgboost还可以支持线性分类器，这个时候xgboost相当于带L1和L2正则化项的logistic回归（分类问题）或者线性回归（回归问题）。
2. GBDT采用了一阶展开，而xgboost采用了泰勒二阶展开，二阶信息能让梯度收敛更快。 
3. xgboost加入了正则项，用于控制模型的复杂度，降低过拟合风险。正则项惩罚了叶子节点数量和叶子节点的输出值的平方和。
4. xgboost在计算分裂节点属性的时候，可以使用分桶算法，加快计算速度。同时也支持针对二阶导数的加权分桶。
5. xgboost加入了缺失值处理。

系统上：
1. 并行的优化。不同维度特征排序可以同时进行，不同叶子节点的分裂可以同时进行。
2. cache的hit miss优化
3. 使用硬盘处理超大数据集。数据压缩之后再放到内存，用计算换IO时间；数据sharding到多个磁盘，读取时同时使用。

LightGBM对于xgboost的提升主要体现在对于大数据集的处理上。xgboost在面对大数据集的时候，会出现计算速度慢的缺点。因为在选择分裂属性的时候，如果是精确求解，需要扫描每一个属性的每一个取值；即便是使用的近似解法，特征数量很多的时候需要扫描每一个特征。  
所以，LightGBM的在样本维度和特征维度都做了优化。GOSS的采样可以大幅减少样本数，尽量保持梯度绝对值大的样本，同时保持数据分布不变。EFB把互斥特征绑定，减少需要扫描的特征数。

### <font face="微软雅黑" color=#A52A2A > 5. GBDT是如何做分裂的

GBDT只是boosting的一种形式，里面的弱学习器是可以选择多种的，只要满足低方差、高偏差的即可（因为GBDT主要是通过多轮迭代，降低偏差来降低泛化误差）。一般而言选择cart树比较多，所以GBDT如何分裂其实是在问cart如何做分裂的。如果是回归树，那么是用平方差最小来选择特征；如果是分类树，是用gini系数。

### <font face="微软雅黑" color=#A52A2A > 6. GBDT是如何做分类任务

GBDT不管是做分类还是回归任务，里面的基学习器都只能是回归树，因为迭代过程是需要针对残差相加减的，而类别的相加减是没有意义的。在做分类任务的时候，针对每一个可能的类别，都需要训练一棵树。比如在三分类问题中，每一轮我们都会训练三棵树，如果样本的label是第二类别，那么三棵树的理论输出应该是[0,1,0]。第一轮训练三棵树之后，会加上softmax函数，求得三个类别对应的概率，然后用[0,1,0]减去softmax处理之后的概率得到残差，作为第二轮的训练lable。

### <font face="微软雅黑" color=#A52A2A > 7. GBDT是如何产生特征组合

参考Facebook LR+GBDT，待完善

### <font face="微软雅黑" color=#A52A2A > 6. xgboost为什么要二阶展开

1. 二阶信息本身就能让梯度收敛更快更准确，这在牛顿法优化里面已经证明。可以认为一阶导指引梯度方向，二阶导指引梯度方向如何变化。其实xgboost的迭代过程就是牛顿法的过程。牛顿法是用来求解f(x)=0的根的方法，通过不断地迭代，来逼近零点。而f(x)最优化的过程就是求f'(x)=0的过程，我们把f'(x)看成函数，使用牛顿法，就有x(n+1) = x(n)-f'(x)/f"(x)。而-f'(x)/f"(x)就是每一轮树的叶子节点输出。
2. xgboost使用二阶展开可以统一形式，只要损失函数是二次可微的，能够给出g和h，那么就可以进行计算，这样会方便自定义损失函数。

### <font face="微软雅黑" color=#A52A2A > 7. 牛顿法与SGD区别

牛顿法是用来求解f(x)=0的根的方法，我们把泰勒公式展开到二阶，对其求导，然后令f'(x)=0，得到的表达式x(n+1)=x(n)-f'(x)/f"(x)。这能让梯度收敛更快更准确。而SGD只是指明了梯度方向。

### <font face="微软雅黑" color=#A52A2A > 8. 牛顿法能用于非凸函数吗

不能，梯度下降和牛顿法停止的条件都是f'(x)=0。梯度下降方法使用的是负梯度方向，这个方向对应的值是下降的，它有可能会被trap在一个local minima，但是它不会往高处走；但是牛顿法不一样，它变化的方向是-f'(x)/f"(x)，这里的f"(x)是不定的，它有可能会使函数的迭代走到极大值方向。

### <font face="微软雅黑" color=#A52A2A > 9. ID3, C4.5, CART是什么，有什么优势

### <font face="微软雅黑" color=#A52A2A > 10. 决策树是如何剪枝的